In [2]:
from nltk.corpus import sentiwordnet as swn 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import re
import pymongo
#from pymongo import MongoClient 
from nltk.tokenize import TweetTokenizer 
import nltk
from nltk.tokenize import PunktSentenceTokenizer
import collections
import operator
from nltk.util import ngrams
import datetime 
import numpy as np
from nltk.corpus import sentiwordnet as swn 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import re
import pymongo
#from pymongo import MongoClient 
from nltk.tokenize import TweetTokenizer 
import nltk
from nltk.tokenize import PunktSentenceTokenizer
import collections
import operator
from nltk.util import ngrams
import json
import re
import os
from datetime import datetime, timedelta
client = pymongo.MongoClient('mongodb://TwitterIPO?authSource=TwitterIPO')

PriceData = client.TwitterIPO.PriceData
tweetwordvector=client.TwitterIPO.CompanyTweetWordVector
client.TwitterIPO.CombineWordVectorWithPrices.delete_many({})
# Get Price Date and Create a Dataframe
prices = [x for x in PriceData.find({},{'Name':1,'Date':1, 'Volume':1,'direction':1})]
df_prices = pd.DataFrame(prices)
df_prices= df_prices.rename(columns={'Name': 'IPO_Company','Volume':'Stock_Volume','direction':'Stock_Direction'})
df_prices['Price_Date'] = pd.to_datetime(df_prices.Date)
# Format the date
df_prices['Price_Date'] = df_prices.Price_Date.apply(lambda x: datetime.utcfromtimestamp(x.value / 1e9).date())

tweetdf=pd.DataFrame(list(tweetwordvector.find({})))

#tweetdf['Date'] = tweetdf.date.apply(lambda x: datetime.utcfromtimestamp(x.value / 1e9).date())
tweetdf['Price_Date'] = tweetdf.price_date.map(lambda x: x + timedelta(days=1))
tweetdf['Price_Date'] =tweetdf.price_date.map(lambda x: x.date())

tweetdf= tweetdf.rename(columns={'ipo_company': 'IPO_Company'})
df_combined = pd.merge(df_prices,tweetdf, on=['Price_Date','IPO_Company'], how='inner')
df_combined = df_combined.dropna(how='any')
df_combined.Price_Date = df_combined.Price_Date.map(lambda t: datetime(t.year, t.month, t.day))

client.TwitterIPO.CombineWordVectorWithPrices.insert_many(df_combined.to_dict('records'))




0       2016-05-12
1       2016-06-03
2       2016-06-14
3       2016-06-15
4       2016-06-16
5       2016-10-04
6       2016-10-05
7       2016-10-06
8       2016-10-08
9       2016-10-15
10      2016-10-18
11      2016-10-19
12      2016-10-21
13      2016-10-23
14      2016-10-24
15      2016-10-25
16      2016-10-27
17      2016-10-03
18      2016-11-03
19      2016-11-04
20      2016-11-05
21      2016-11-07
22      2016-11-09
23      2016-08-08
24      2016-08-11
25      2016-08-13
26      2016-08-15
27      2016-08-26
28      2016-09-11
29      2016-09-12
           ...    
1661    2016-04-23
1662    2016-04-24
1663    2016-04-25
1664    2016-04-26
1665    2016-04-27
1666    2016-04-28
1667    2016-04-29
1668    2016-04-30
1669    2016-05-02
1670    2016-05-03
1671    2016-05-04
1672    2016-05-05
1673    2016-05-09
1674    2016-05-11
1675    2016-05-12
1676    2016-05-18
1677    2016-05-19
1678    2016-05-20
1679    2016-05-23
1680    2016-05-24
1681    2016-05-25
1682    2016